# Glow-TTS Training 01

## Google Drive

In [1]:
# mount google drive for data storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Install Coqui TTS

In [2]:
# upgrade pip, install tts, and upgrade tensorflow
! pip install -U pip
! pip install TTS
! pip install -U tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.4 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 62.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.2/15.2 MB 91.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 40.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 87.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 44.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 69.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 22.7 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.0
    Uninstalling numpy-1.22.0:
      Successfully uninstalled numpy-1.22.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successful

## Imports

In [1]:
# import the necessary libraries
import os
import glob

from trainer import Trainer, TrainerArgs

from TTS.utils.audio import AudioProcessor
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.datasets import load_tts_samples
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
from TTS.tts.configs.shared_configs import BaseDatasetConfig

## Load Dataset

In [2]:
# set output path under google drive directory
output_path = "/content/drive/MyDrive/ljspeech-002/tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)

# set the dataset configuration
# ljspeech dataset was chosen, but only one speaker (002) will be used
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="/content/drive/MyDrive/ljspeech-002/metadata.csv", path="/content/drive/MyDrive/ljspeech-002"
)

## Config

In [20]:
# model architecture and configuration
# 250 epochs will be trained with 32 training batch size
# checkpoints every 200 steps, meaning around every 19 epochs,
# since 337 training samples / 32 batch size = 10.5 steps per epoch
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=250,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=200,
)

## Audio Processor

In [21]:
# audio processor for feature extraction and audio input/output
ap = AudioProcessor.init_from_config(config)

 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


## Tokenizer

In [22]:
# tokenizer for converting text to sequences of token ids
tokenizer, config = TTSTokenizer.init_from_config(config)

## Load Data Samples

In [23]:
# load the training and evaluation samples from the dataset
# each sample is a list of [text, audio_file_path, speaker_name]
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 337 files in /content/drive/MyDrive/ljspeech-002


## Model Initialization

In [24]:
# model initialization with configuration, audio processor, tokenizer, and no speaker manager
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# trainer generic api for training the model
trainer = Trainer(
    TrainerArgs(),
    config,
    output_path,
    model=model,
    train_samples=train_samples,
    eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000

 > Model has 28610257 parameters


## Training

In [ ]:
# start the training process
trainer.fit()


 > EPOCH: 0/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(

 > TRAINING (2024-06-19 17:47:59) 




> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 334
 | > Preprocessing samples
 | > Max text length: 179
 | > Min text length: 32
 | > Avg text length: 107.95808383233533
 | 
 | > Max audio length: 222643.0
 | > Min audio length: 33971.0
 | > Avg audio length: 146823.3113772455
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.


/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()

   --> TIME: 2024-06-19 17:48:06 -- STEP: 0/11 -- GLOBAL_STEP: 0
     | > current_lr: 2.5e-07 
     | > step_time: 1.6499  (1.6499032974243164)
     | > loader_time: 4.9393  (4.939289569854736)

 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned `None` outputs. Skipping training step.
 [!] `train_step()` retuned 



> DataLoader initialization
| > Tokenizer:
	| > add_blank: False
	| > use_eos_bos: False
	| > use_phonemes: True
	| > phonemizer:
		| > phoneme language: en-us
		| > phoneme backend: gruut
| > Number of instances : 3
 | > Preprocessing samples
 | > Max text length: 148
 | > Min text length: 85
 | > Avg text length: 125.66666666666667
 | 
 | > Max audio length: 208051.0
 | > Min audio length: 103603.0
 | > Avg audio length: 168029.66666666666
 | > Num. instances discarded samples: 0
 | > Batch group size: 0.
 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.5940234661102295 (+0)
     | > avg_loss: 3.2766103744506836 (+0)
     | > avg_log_mle: 0.7733529210090637 (+0)
     | > avg_loss_dur: 2.5032575130462646 (+0)

 > BEST MODEL : /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000/best_model_11.pth

 > EPOCH: 1/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000

 > TRAINING (2024-06-19 17:48:25) 

 > EVALUATION 



...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.7235536575317383 (-0.8521113395690918)
     | > avg_loss: 0.1895299255847931 (+0.018393486738204956)
     | > avg_log_mle: -0.11278283596038818 (-0.0015549659729003906)
     | > avg_loss_dur: 0.3023127615451813 (+0.019948452711105347)


 > EPOCH: 215/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000

 > TRAINING (2024-06-19 20:08:43) 

   --> TIME: 2024-06-19 20:09:03 -- STEP: 10/11 -- GLOBAL_STEP: 2375
     | > loss: 0.1673840582370758  (0.1810850590467453)
     | > log_mle: -0.16402220726013184  (-0.1470654010772705)
     | > loss_dur: 0.33140626549720764  (0.32815046012401583)
     | > amp_scaler: 32768.0  (32768.0)
     | > grad_norm: tensor(1.8118, device='cuda:0')  (tensor(2.8061, device='cuda:0'))
     | > current_lr: 5.375e-05 
     | > step_time: 0.416  (1.4584625959396362)
     | > loader_time: 0.0055  (0.025617575645446776)


 > EVALUATION 



 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.713526725769043 (-0.010026931762695312)
     | > avg_loss: 0.18460598587989807 (-0.0049239397048950195)
     | > avg_log_mle: -0.1147911548614502 (-0.0020083189010620117)
     | > avg_loss_dur: 0.29939714074134827 (-0.002915620803833008)


 > EPOCH: 216/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000

 > TRAINING (2024-06-19 20:09:20) 

 > EVALUATION 



 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.6896748542785645 (-0.023851871490478516)
     | > avg_loss: 0.18584445118904114 (+0.0012384653091430664)
     | > avg_log_mle: -0.11471962928771973 (+7.152557373046875e-05)
     | > avg_loss_dur: 0.30056408047676086 (+0.0011669397354125977)


 > EPOCH: 217/250
 --> /content/drive/MyDrive/ljspeech-002/tts_train_dir/run-June-19-2024_05+47PM-0000000

 > TRAINING (2024-06-19 20:09:56) 
